In [ ]:
#recheck dependencies 

In [1]:
import panel as pn
import numpy as np
from pycromanager import Core
import xarray as xr ## used instead of np for RGBA support
import hvplot.xarray
import holoviews as hv

pn.extension()

In [2]:
core = Core()

In [5]:
image_button = pn.widgets.Button(name='Snap Image', button_type='primary')
#figure instead of curve
image_widget = pn.panel(hv.Curve([]))
histo_button = pn.widgets.Button(name='Histogram', button_type='primary')
histo_widget=pn.panel(hv.Curve([]),)


up_button    = pn.widgets.Button(name='^', button_type='primary')
left_button  = pn.widgets.Button(name='<', button_type='primary')
right_button = pn.widgets.Button(name='>', button_type='primary')
down_button  = pn.widgets.Button(name='v', button_type='primary')
coord_text   = pn.widgets.TextInput(value=str([core.get_x_position(),core.get_y_position()]))
step_size    = pn.widgets.FloatInput(name='step size', value=1., step=1e-1, start=0, end=1500)

xy_stage=core.get_xy_stage_device()

def snap():
    core.wait_for_system()
    core.snap_image()
    tagged_image = core.get_tagged_image()
    
    return tagged_image
                                

def on_image_button_clicked(b):
    tagged_image=snap()
    
    height = tagged_image.tags['Height']
    width = tagged_image.tags['Width']
    pixels = np.reshape(tagged_image.pix,
                    newshape=[height, width, tagged_image.pix.shape[0]//(height*width)])
    
    #reshape and stay as np
    x=list(range(pixels.shape[0]))
    y=list(range(pixels.shape[1]))
    chan=list(range(pixels.shape[2]))

    coords={'x':x,
        'y':y,
        'chan':chan}
    
    pix_array=xr.DataArray(pixels,coords=coords)
    image_widget.object=pix_array.hvplot.rgb(x='y',y='x',bands='chan',data_aspect=1,flip_yaxis=True)
                                             
    
def on_histo_button_clicked(b):
    tagged_image= snap()
    hist=xr.DataArray(tagged_image.pix,name='histogram')
    histo_widget.object=hist.hvplot.hist(bins=64) # 
    
def on_move_up_button_clicked(b):
    core.set_relative_xy_position(xy_stage, 0,step_size.value)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])
    
def on_move_left_button_clicked(b):
    core.set_relative_xy_position(xy_stage, -step_size.value,0)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])

def on_move_right_button_clicked(b):
    core.set_relative_xy_position(xy_stage, step_size.value,0)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])
    
def on_move_down_button_clicked(b):
    core.set_relative_xy_position(xy_stage, 0,-step_size.value)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])


image_button.on_click(on_image_button_clicked)
histo_button.on_click(on_histo_button_clicked)
up_button.on_click(on_move_up_button_clicked)
left_button.on_click(on_move_left_button_clicked) 
right_button.on_click(on_move_right_button_clicked)
down_button.on_click(on_move_down_button_clicked)


app = pn.Column(
    pn.Row(
        pn.Column(
            image_button, 
            histo_button, 
            step_size,
            pn.Row(up_button,align='center'),
            pn.Row(left_button, right_button, align='center'),
            pn.Row(down_button, align='center'),
        coord_text, sizing_mode='stretch_height'),
        image_widget),
    histo_widget)


In [6]:
tabs = pn.Tabs(('Display Image',app), dynamic=True)
tabs.servable()

Tabs(dynamic=True)
    [0] Column
        [0] Row
            [0] Column(sizing_mode='stretch_height')
                [0] Button(button_type='primary', name='Snap Image')
                [1] Button(button_type='primary', name='Histogram')
                [2] FloatInput(end=1500, name='step size', start=0, value=1.0)
                [3] Row(align='center')
                    [0] Button(button_type='primary', name='^')
                [4] Row(align='center')
                    [0] Button(button_type='primary', name='<')
                    [1] Button(button_type='primary', name='>')
                [5] Row(align='center')
                    [0] Button(button_type='primary', name='v')
                [6] TextInput(value='[6044.4400000000005, ...)
            [1] HoloViews(Curve, height=300, sizing_mode='fixed', width=300)
        [1] HoloViews(Curve, height=300, sizing_mode='fixed', width=300)